# Using Word2Vec

In [2]:
import pandas as pd # provide sql-like data manipulation tools. very handy.
pd.options.mode.chained_assignment = None
import numpy as np # high dimensional vector computing library.
from copy import deepcopy
from string import punctuation
from random import shuffle
from keras.models import Sequential
import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below
import keras
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from keras.layers import Activation, Dense
from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
from sklearn.preprocessing import scale
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.sampledata.periodic_table import elements

# Split Data into Training and Testing set¶

In [3]:
import pandas as pd
import re, string
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
import traceback
import re, string
def clean_str(string):
  """
  String cleaning before vectorization
  """
  try:
    string = re.sub(r'^https?:\/\/<>.*[\r\n]*', '', string, flags=re.MULTILINE)
    #string = re.sub(r"[^A-Za-z]", " ", string) 
    string = re.sub(r"\W+", " ", string)
    words = string.strip().lower().split()    
    words = [w for w in words if len(w)>1]
    #return words list
    return words
  except:
    print (traceback.print_exc())
    return ""

In [5]:
#Load training and Test data
import pandas as pd
data=pd.read_csv('tweets_sentiment.txt', header=0, delimiter="\t", quoting=3)

FileNotFoundError: [Errno 2] File b'tweets_sentiment.txt' does not exist: b'tweets_sentiment.txt'

In [6]:
data.head()

NameError: name 'data' is not defined

In [194]:
df=data['text']

In [195]:
a=df[df.str.contains('macron' or 'emmanuelmacron' or 'enmarche' or '@emmanuelmacron' or '#emmanuelmacron' or 'mlpofficiel' or 'mlp_officiel' or 'lepen' or 'marine2017' or '@mlp_officiel' or 'mlp',na=False)]

In [196]:
len(a)

574555

In [197]:
data['text2']=a

In [198]:
data=data.dropna()

In [199]:
data.head()

,text,sentiment,text2
2,ah macron et ses approximations,1.0,ah macron et ses approximations
9,a marseille fillon cible macron lepen et méle...,1.0,a marseille fillon cible macron lepen et méle...
16,face mélenchon et son programme de destruction...,1.0,face mélenchon et son programme de destruction...
18,macron m a fait réfléchir sur quand mélenchon...,1.0,macron m a fait réfléchir sur quand mélenchon...
39,sérieusement venant des soutiens de macron l é...,1.0,sérieusement venant des soutiens de macron l é...


In [200]:
data['text']=data['text2']

In [201]:
del data['text2']

In [202]:
data.head()

,text,sentiment
2,ah macron et ses approximations,1.0
9,a marseille fillon cible macron lepen et méle...,1.0
16,face mélenchon et son programme de destruction...,1.0
18,macron m a fait réfléchir sur quand mélenchon...,1.0
39,sérieusement venant des soutiens de macron l é...,1.0


In [203]:
data['cleanText'] = data['text'].apply(clean_str)
data.sentiment=data.sentiment.astype(int)

In [204]:
data['Sentiment']=data['sentiment']

In [205]:
data['SentimentText']=data['text']

In [206]:
data['tokens']=data['cleanText']

In [207]:
data.head()

,text,sentiment,cleanText,Sentiment,SentimentText,tokens
2,ah macron et ses approximations,1,"[ah, macron, et, ses, approximations]",1,ah macron et ses approximations,"[ah, macron, et, ses, approximations]"
9,a marseille fillon cible macron lepen et méle...,1,"[marseille, fillon, cible, macron, lepen, et, ...",1,a marseille fillon cible macron lepen et méle...,"[marseille, fillon, cible, macron, lepen, et, ..."
16,face mélenchon et son programme de destruction...,1,"[face, lenchon, et, son, programme, de, destru...",1,face mélenchon et son programme de destruction...,"[face, lenchon, et, son, programme, de, destru..."
18,macron m a fait réfléchir sur quand mélenchon...,1,"[macron, fait, fl, chir, sur, quand, lenchon, ...",1,macron m a fait réfléchir sur quand mélenchon...,"[macron, fait, fl, chir, sur, quand, lenchon, ..."
39,sérieusement venant des soutiens de macron l é...,1,"[rieusement, venant, des, soutiens, de, macron...",1,sérieusement venant des soutiens de macron l é...,"[rieusement, venant, des, soutiens, de, macron..."


In [208]:
# to delete unwanted columns
del data['cleanText']
del data['sentiment']
del data['text']

In [209]:
data.head()

,Sentiment,SentimentText,tokens
2,1,ah macron et ses approximations,"[ah, macron, et, ses, approximations]"
9,1,a marseille fillon cible macron lepen et méle...,"[marseille, fillon, cible, macron, lepen, et, ..."
16,1,face mélenchon et son programme de destruction...,"[face, lenchon, et, son, programme, de, destru..."
18,1,macron m a fait réfléchir sur quand mélenchon...,"[macron, fait, fl, chir, sur, quand, lenchon, ..."
39,1,sérieusement venant des soutiens de macron l é...,"[rieusement, venant, des, soutiens, de, macron..."


In [210]:
data.tail()

,Sentiment,SentimentText,tokens
1086909,1,pol is getting closer to blowing the macronga...,"[pol, is, getting, closer, to, blowing, the, m..."
1086912,-1,macron ne veut pas se donner les moyens de lu...,"[macron, ne, veut, pas, se, donner, les, moyen..."
1086913,1,because he hasn t backed enough losers over t...,"[because, he, hasn, backed, enough, losers, ov..."
1086914,1,the kiss of death for macron obama,"[the, kiss, of, death, for, macron, obama]"
1086915,1,concernant les propos de macron sur la refonda...,"[concernant, les, propos, de, macron, sur, la,..."


In [211]:
len(data)

574555

In [212]:
data = data.reset_index(drop=True)

In [213]:
data.tail()

,Sentiment,SentimentText,tokens
574550,1,pol is getting closer to blowing the macronga...,"[pol, is, getting, closer, to, blowing, the, m..."
574551,-1,macron ne veut pas se donner les moyens de lu...,"[macron, ne, veut, pas, se, donner, les, moyen..."
574552,1,because he hasn t backed enough losers over t...,"[because, he, hasn, backed, enough, losers, ov..."
574553,1,the kiss of death for macron obama,"[the, kiss, of, death, for, macron, obama]"
574554,1,concernant les propos de macron sur la refonda...,"[concernant, les, propos, de, macron, sur, la,..."


In [214]:
d1=data.head(500)

In [215]:
d2=data.tail(500)

In [118]:
# Exporting Data For Manual Validation:
writer = pd.ExcelWriter('vld.xlsx')
d1.to_excel(writer,'Sheet1')
d2.to_excel(writer,'Sheet2')
writer.save()

In [216]:
data=data[500:574054]

In [217]:
data.tail()

,Sentiment,SentimentText,tokens
574049,1,l escroquerie du vote macron via,"[escroquerie, du, vote, macron, via]"
574050,-1,jevotepour macron car je fais partie des crét...,"[jevotepour, macron, car, je, fais, partie, de..."
574051,1,les soutiens de m macron agissent dans la vio...,"[les, soutiens, de, macron, agissent, dans, la..."
574052,1,macron liberalisme pas gauche xtreme droite a...,"[macron, liberalisme, pas, gauche, xtreme, dro..."
574053,1,as obama implored nigerians to vote right as ...,"[as, obama, implored, nigerians, to, vote, rig..."


In [218]:
n=1000000
x_train, x_test, y_train, y_test = train_test_split(np.array(data.head(n).tokens),np.array(data.head(n).Sentiment), test_size=0.2)

In [219]:
x_train

array([ list(['malgr', 'son', 'retrait', 'de', 'la', 'vie', 'politique', 'barack', 'obama', 'affich', 'officiellement', 'sa', 'confiance', 'emmanuel', 'macron', 'dans']),
       list(['neelie', 'kroes', 'la', 'dirigeante', 'une', 'entreprise', 'offshore', 'aux', 'bahamas', 'rencontr', 'macron', 'lasvegas', 'en', 'ja']),
       list(['macron']), ...,
       list(['attali', 'le', 'mentor', 'de', 'macron', 'vous', 'explique', 'son', 'projet', 'pour', 'la', 'france']),
       list(['macron', 'rothschild', 'banker', 'bilderberg', 'member', 'bureaucrat', 'in', 'hollande', 'government', 'media', 'he', 'an', 'outsider', 'this', 'is', 'lite']),
       list(['mme', 'lepen', 'que', 'proposez', 'vous', 'pour', 'duire', 'le', 'ch', 'mage', 'regardez', 'est', 'le', 'score', 'sur', 'candy', 'crush', 'de', 'monsieur', 'macron'])], dtype=object)

In [220]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')
x_test = labelizeTweets(x_test, 'TEST')

458843it [00:02, 156452.57it/s]
114711it [00:01, 70520.78it/s]


In [221]:
x_train[0]

TaggedDocument(words=['malgr', 'son', 'retrait', 'de', 'la', 'vie', 'politique', 'barack', 'obama', 'affich', 'officiellement', 'sa', 'confiance', 'emmanuel', 'macron', 'dans'], tags=['TRAIN_0'])

### build the word2vec model from x_train i.e. the corpus

In [222]:
frenchW2V = gensim.models.KeyedVectors.load_word2vec_format('frWiki.bin', binary=True)

2017-08-24 11:29:59,988 : INFO : loading projection weights from frWiki.bin
2017-08-24 11:30:05,740 : INFO : loaded (66274, 1000) matrix from frWiki.bin


In [223]:
frenchW2V.wv.syn0.shape

(66274, 1000)

In [224]:
frenchW2V['bien']

array([  3.92278284e-02,  -1.21879382e-02,  -5.50427884e-02,
         5.55902123e-02,   6.10539652e-02,   9.64832902e-02,
        -1.28307529e-02,  -1.15820607e-02,  -2.21761074e-02,
        -9.99968778e-03,   2.94949394e-04,  -1.47818178e-02,
        -7.75420740e-02,   2.75168717e-02,   3.78509611e-02,
        -1.96574461e-02,   5.44788241e-02,   5.37251383e-02,
        -2.15995288e-03,   2.10342873e-02,  -1.45163193e-01,
         9.81824193e-03,   8.27191770e-02,   2.56233085e-02,
        -1.69443563e-02,   6.22188821e-02,  -4.27901261e-02,
         8.07462446e-03,  -1.06191412e-01,  -2.75264215e-03,
        -1.54654197e-02,   4.38642316e-03,  -1.45791203e-01,
        -9.34110880e-02,  -4.71170284e-02,  -1.21728517e-02,
        -4.58486751e-02,   6.07920513e-02,   1.63038164e-01,
         3.35928723e-02,   1.49394006e-01,   1.34771377e-01,
        -1.79264750e-02,   3.60278673e-02,  -3.77434725e-03,
         2.10834350e-02,  -4.23749583e-03,  -3.64756063e-02,
        -4.60192896e-02,

In [225]:
frenchW2V.most_similar('mal')

2017-08-24 11:30:16,387 : INFO : precomputing L2-norms of word weight vectors


[(u'bien', 0.5377739667892456),
 (u'mieux', 0.4622610807418823),
 (u'pourquoi', 0.3916780352592468),
 (u'peur', 0.3831149637699127),
 (u'pourtant', 0.3812871277332306),
 (u'foutu', 0.3798130452632904),
 (u'compliqu\xe9', 0.37696385383605957),
 (u'intentionn\xe9s', 0.36648163199424744),
 (u'comprendre', 0.36104655265808105),
 (u'trop', 0.3609797954559326)]

In [226]:
print 'building tf-idf matrix ...'
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in x_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print 'vocab size :', len(tfidf)

building tf-idf matrix ...
vocab size : 15482


In [227]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += frenchW2V[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [228]:
n_dim=1000
from sklearn.preprocessing import scale
train_vecs_w2vfr = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2vfr = scale(train_vecs_w2vfr)

test_vecs_w2vfr = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2vfr = scale(test_vecs_w2vfr)

100%|██████████| 114711/114711 [00:28<00:00, 3993.67it/s]


In [42]:
# Saving numpy array
np.savetxt('trainw2vfr.txt',train_vecs_w2vfr , fmt='%10.8f')
np.savetxt('testw2vfr.txt',test_vecs_w2vfr , fmt='%10.8f')
np.savetxt('y_trainfr.txt',y_train , fmt='%d')
np.savetxt('y_testfr.txt',y_test , fmt='%d')

## Running the model on the Train and Test Data:

In [43]:
import pandas as pd # provide sql-like data manipulation tools. very handy.
pd.options.mode.chained_assignment = None
import numpy as np # high dimensional vector computing library.
from copy import deepcopy
from string import punctuation
from random import shuffle
from keras.models import Sequential
import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below
import keras
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from keras.layers import Activation, Dense
from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
from sklearn.preprocessing import scale

In [44]:
# importing numpy arrays
trainvecs = np.loadtxt('trainw2vfr.txt', dtype=float)
testvecs = np.loadtxt('testw2vfr.txt', dtype=float)
y_trainvecs = np.loadtxt('y_trainfr.txt', dtype=int)
y_testvecs = np.loadtxt('y_testfr.txt', dtype=int)

## Using ANN

In [45]:
# This model works better than the model below
# to define that my model is a sequence of layers
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph/ANN', histogram_freq=0, write_graph=True, write_images=True)
model = Sequential()
# defining my first hidden layer, output size is 100
model.add(Dense(100, activation='sigmoid', input_dim=1000))
# defining my second hidden layer, output size is 50
model.add(Dense(50, activation='relu', input_dim=100))
# defining my third hidden layer, output size is 32
model.add(Dense(32, activation='relu', input_dim=50))
# defining my fourth hidden layer, output size is 16
model.add(Dense(16, activation='relu', input_dim=32))
# defining my output layer, output size is 1
model.add(Dense(1, activation='sigmoid'))

# I just need to compile the model and I'll be ready to train it. 
#When I compile the model, I declare the loss function and the optimizer (SGD, Adam, etc.).
# Here I have used 'Adam' optimizer
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [46]:
# To fit the model, all I have to do is declare the batch size and number of epochs to train for, 
# then pass in my training data
model.fit(trainvecs, y_trainvecs, epochs=125, batch_size=32, verbose=2,callbacks=[tbCallBack])

Epoch 1/125
66s - loss: -1.3933e+00 - acc: 0.8168
Epoch 2/125
64s - loss: -1.6835e+00 - acc: 0.8268
Epoch 3/125
63s - loss: -1.7801e+00 - acc: 0.8294
Epoch 4/125
63s - loss: -1.8223e+00 - acc: 0.8306
Epoch 5/125
64s - loss: -1.8388e+00 - acc: 0.8306
Epoch 6/125
64s - loss: -1.8607e+00 - acc: 0.8315
Epoch 7/125
63s - loss: -1.8763e+00 - acc: 0.8329
Epoch 8/125
63s - loss: -1.8928e+00 - acc: 0.8323
Epoch 9/125
63s - loss: -1.9105e+00 - acc: 0.8324
Epoch 10/125
64s - loss: -1.9109e+00 - acc: 0.8325
Epoch 11/125
63s - loss: -1.9189e+00 - acc: 0.8324
Epoch 12/125
64s - loss: -1.9071e+00 - acc: 0.8316
Epoch 13/125
63s - loss: -1.9261e+00 - acc: 0.8329
Epoch 14/125
64s - loss: -1.9341e+00 - acc: 0.8329
Epoch 15/125
64s - loss: -1.9453e+00 - acc: 0.8334
Epoch 16/125
64s - loss: -1.9413e+00 - acc: 0.8327
Epoch 17/125
63s - loss: -1.9504e+00 - acc: 0.8328
Epoch 18/125
64s - loss: -1.9443e+00 - acc: 0.8327
Epoch 19/125
63s - loss: -1.9611e+00 - acc: 0.8329
Epoch 20/125
63s - loss: -1.9593e+00 - a

In [47]:
# Saving the model:
from keras.models import load_model
import h5py
model.save('ann_model_fr.h5')

#### Executing ANN Model 

In [48]:
# Calling/ importing the model
model = load_model('ann_model_fr.h5')

In [49]:
score = model.evaluate(testvecs, y_testvecs, batch_size=128, verbose=2)
print score[1]

0.831637767968


#  Using CNN

In [146]:
# CNN for the sentiment analysis problem
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [147]:
# load the dataset but only keep the top n words, zero the rest
top_words = 66274

In [148]:
# pad dataset to a maximum review length in words
max_words = 140

In [149]:
X_train=trainvecs

In [150]:
X_test=testvecs

In [151]:
y_train=y_trainvecs

In [152]:
y_test=y_testvecs

In [153]:
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [154]:
# create the model
model = Sequential()
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph/CNN', histogram_freq=0, write_graph=True, write_images=True)
model.add(Embedding(top_words, 32, input_length=max_words))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
#model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='sigmoid'))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='tanh'))
model.add(MaxPooling1D(pool_size=2))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(125, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 140, 32)           2120768   
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 140, 32)           3104      
_________________________________________________________________
conv1d_27 (Conv1D)           (None, 140, 32)           3104      
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 70, 32)            0         
_________________________________________________________________
max_pooling1d_20 (MaxPooling (None, 35, 32)            0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 1120)              0         
_________________________________________________________________
dense_35 (Dense)             (None, 250)               280250    
__________

In [155]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=128, verbose=2,callbacks=[tbCallBack])

Train on 458843 samples, validate on 114711 samples
Epoch 1/100
45s - loss: -7.3660e-01 - acc: 0.7792 - val_loss: -1.1727e+00 - val_acc: 0.8002
Epoch 2/100
45s - loss: -1.3613e+00 - acc: 0.8007 - val_loss: -1.3425e+00 - val_acc: 0.7991
Epoch 3/100
45s - loss: -1.5244e+00 - acc: 0.8062 - val_loss: -1.4022e+00 - val_acc: 0.7946
Epoch 4/100
45s - loss: -1.6254e+00 - acc: 0.8094 - val_loss: -1.4274e+00 - val_acc: 0.8102
Epoch 5/100
45s - loss: -1.6942e+00 - acc: 0.8113 - val_loss: -1.4688e+00 - val_acc: 0.8051
Epoch 6/100
45s - loss: -1.7595e+00 - acc: 0.8135 - val_loss: -1.4741e+00 - val_acc: 0.7990
Epoch 7/100
45s - loss: -1.8042e+00 - acc: 0.8147 - val_loss: -1.4671e+00 - val_acc: 0.8009
Epoch 8/100
45s - loss: -1.8474e+00 - acc: 0.8160 - val_loss: -1.4759e+00 - val_acc: 0.8039
Epoch 9/100
45s - loss: -1.8848e+00 - acc: 0.8174 - val_loss: -1.4647e+00 - val_acc: 0.8082
Epoch 10/100
46s - loss: -1.9122e+00 - acc: 0.8184 - val_loss: -1.4524e+00 - val_acc: 0.8015
Epoch 11/100
46s - loss: -1

45s - loss: -2.1207e+00 - acc: 0.8299 - val_loss: -1.2074e+00 - val_acc: 0.8112
Epoch 90/100
45s - loss: -2.1227e+00 - acc: 0.8300 - val_loss: -1.2105e+00 - val_acc: 0.8141
Epoch 91/100
46s - loss: -2.1192e+00 - acc: 0.8297 - val_loss: -1.2132e+00 - val_acc: 0.8144
Epoch 92/100
45s - loss: -2.1224e+00 - acc: 0.8301 - val_loss: -1.2019e+00 - val_acc: 0.8111
Epoch 93/100
46s - loss: -2.1167e+00 - acc: 0.8296 - val_loss: -1.1692e+00 - val_acc: 0.8161
Epoch 94/100
45s - loss: -2.1200e+00 - acc: 0.8299 - val_loss: -1.1805e+00 - val_acc: 0.8148
Epoch 95/100
46s - loss: -2.1104e+00 - acc: 0.8294 - val_loss: -1.1698e+00 - val_acc: 0.8132
Epoch 96/100
45s - loss: -2.1133e+00 - acc: 0.8293 - val_loss: -1.1722e+00 - val_acc: 0.8116
Epoch 97/100
45s - loss: -2.1135e+00 - acc: 0.8295 - val_loss: -1.1839e+00 - val_acc: 0.8125
Epoch 98/100
45s - loss: -2.1226e+00 - acc: 0.8301 - val_loss: -1.2070e+00 - val_acc: 0.8126
Epoch 99/100
46s - loss: -2.1244e+00 - acc: 0.8303 - val_loss: -1.1940e+00 - val_ac

In [156]:
# Saving the model
from keras.models import load_model
import h5py
model.save('cnn_model_fr.h5')

In [157]:
# Calling/ Importing the model
model = load_model('cnn_model_fr.h5')

In [158]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 81.34%


## Using LSTM

In [171]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import traceback
import re, string

In [172]:
def clean_str(string):
  """
  String cleaning before vectorization
  """
  try:
    string = re.sub(r'^https?:\/\/<>.*[\r\n]*', '', string, flags=re.MULTILINE)
    string = re.sub(r"[^a-zA-ZÀ-ÿ0-9]", " ", string) 
    #string = re.sub(r"\W+", " ", string)   
    #return words list
    return string
  except:
    print (traceback.print_exc())
    return ""

In [173]:
#Load training and Test data
import pandas as pd
data=pd.read_csv('tweets_sentiment.txt', header=0, delimiter="\t", quoting=3)

In [174]:
df=data['text']
a=df[df.str.contains('macron' or 'emmanuelmacron' or 'enmarche' or '@emmanuelmacron' or '#emmanuelmacron' or 'mlpofficiel' or 'mlp_officiel' or 'lepen' or 'marine2017' or '@mlp_officiel' or 'mlp',na=False)]
data['text2']=a
data=data.dropna()
data['text']=data['text2']
del data['text2']
data['text'] = data['text'].apply(clean_str)
data.sentiment=data.sentiment.astype(int)

In [175]:
data.head()

,text,sentiment
2,ah macron et ses approximations,1
9,a marseille fillon cible macron lepen et méle...,1
16,face mélenchon et son programme de destruction...,1
18,macron m a fait réfléchir sur quand mélenchon...,1
39,sérieusement venant des soutiens de macron l é...,1


In [176]:
data = data.reset_index(drop=True)
data=data[500:574054]

In [177]:
data.head()

,text,sentiment
500,l agitation médiatique autour de macron et de...,1
501,simarinenestpasausecondtour on aura ce genre ...,1
502,sondage macron et le pen en baisse mais toujou...,1
503,simarinenestpasausecondtour on aura ce genre ...,1
504,racisme islam cdanslair islamophobie avecmari...,1


In [178]:
data['text'] = data['text'].apply(lambda x: x.lower())

In [179]:
data.head()

,text,sentiment
500,l agitation médiatique autour de macron et de...,1
501,simarinenestpasausecondtour on aura ce genre ...,1
502,sondage macron et le pen en baisse mais toujou...,1
503,simarinenestpasausecondtour on aura ce genre ...,1
504,racisme islam cdanslair islamophobie avecmari...,1


In [180]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [181]:
embed_dim = 140
lstm_out = 196

model = Sequential()
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph/LSTM', histogram_freq=0, write_graph=True, write_images=True)
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1], dropout=0.2))
#model.add(keras.layers.SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 39, 140)           280000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               264208    
_________________________________________________________________
dense_38 (Dense)             (None, 2)                 394       
Total params: 544,602
Trainable params: 544,602
Non-trainable params: 0
_________________________________________________________________
None


In [169]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

((384281, 39), (384281, 2))
((189273, 39), (189273, 2))


In [183]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 17, batch_size=batch_size, verbose = 2,callbacks=[tbCallBack])

Epoch 1/17
848s - loss: 0.0360 - acc: 0.9871
Epoch 2/17
855s - loss: 0.0363 - acc: 0.9872
Epoch 3/17
855s - loss: 0.0360 - acc: 0.9873
Epoch 4/17
851s - loss: 0.0362 - acc: 0.9871
Epoch 5/17
851s - loss: 0.0367 - acc: 0.9868
Epoch 6/17
850s - loss: 0.0372 - acc: 0.9866
Epoch 7/17
849s - loss: 0.0382 - acc: 0.9864
Epoch 8/17
847s - loss: 0.0379 - acc: 0.9863
Epoch 9/17
843s - loss: 0.0384 - acc: 0.9863
Epoch 10/17
847s - loss: 0.0384 - acc: 0.9862
Epoch 11/17
843s - loss: 0.0395 - acc: 0.9858
Epoch 12/17
845s - loss: 0.0397 - acc: 0.9859
Epoch 13/17
847s - loss: 0.0399 - acc: 0.9858
Epoch 14/17
846s - loss: 0.0402 - acc: 0.9856
Epoch 15/17
847s - loss: 0.0405 - acc: 0.9854
Epoch 16/17
847s - loss: 0.0400 - acc: 0.9858
Epoch 17/17
846s - loss: 0.0410 - acc: 0.9854


In [184]:
# Saving the model
from keras.models import load_model
import h5py
model.save('lstm_model_fr.h5')

#### Executing LSTM Model 

In [185]:
# Calling/ Importing the model
model = load_model('lstm_model_fr.h5')

In [186]:
validation_size = 1600

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.08
acc: 0.98


In [187]:
len(X_validate)

1600

In [188]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
from __future__ import division
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

('pos_acc', 99.63017751479289, '%')
('neg_acc', 93.95161290322581, '%')
